In [50]:
# audio libraries
import librosa
import librosa.display as lplt
import IPython

# import matplotlib to be able to display graphs
import matplotlib.pyplot as plt

# transform .wav into .csv
import csv
import os
import numpy as np
import pandas as pd

# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# model
import keras
import tensorflow as tf
from tensorflow.keras.layers import Normalization
import datetime, os
%load_ext tensorboard
from tensorflow.keras.models import Sequential

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
header = "length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean \
        spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean \
        perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var label".split()
file = open("/kaggle/working/data.csv", "w", newline = "")
with file:
    writer = csv.writer(file)
    writer.writerow(header)
marine_mammals = "AtlanticSpottedDolphin BeardedSeal Beluga_WhiteWhale BlueWhale BottlenoseDolphin Boutu_AmazonRiverDolphin BowheadWhale ClymeneDolphin \
        Commerson'sDolphin CommonDolphin Dall'sPorpoise DuskyDolphin FalseKillerWhale Fin_FinbackWhale FinlessPorpoise Fraser'sDolphin Grampus_Risso'sDolphin \
        GraySeal GrayWhale HarborPorpoise HarbourSeal HarpSeal Heaviside'sDolphin HoodedSeal HumpbackWhale IrawaddyDolphin JuanFernandezFurSeal KillerWhale \
        LeopardSeal Long_FinnedPilotWhale LongBeaked(Pacific)CommonDolphin MelonHeadedWhale MinkeWhale Narwhal NewZealandFurSeal NorthernRightWhale \
        PantropicalSpottedDolphin RibbonSeal RingedSeal RossSeal Rough_ToothedDolphin SeaOtter Short_Finned(Pacific)PilotWhale SouthernRightWhale SpermWhale".split()
for animal in marine_mammals:

    for filename in os.listdir(f"/kaggle/input/best-of-watkins-marine-mammal-sound-database/data/{animal}/"):

        sound_name = f"/kaggle/input/best-of-watkins-marine-mammal-sound-database/data/{animal}/{filename}"
        y, sr = librosa.load(sound_name, mono = True, duration = 30)
        chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
        rmse = librosa.feature.rms(y = y)
        spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
        spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
        rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y = y, sr = sr)
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

        for e in mfcc:
            to_append += f' {np.mean(e)}'

        to_append += f' {animal}'
        file = open('/kaggle/working/data.csv', 'a', newline = '')

        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1912
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=927
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=419
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=507
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=666
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266:

In [25]:
df = pd.read_csv('/kaggle/working/data.csv')
df.shape
os.makedirs(f'/kaggle/working/model_versions', exist_ok=True)
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,label
0,0.540310,0.058862,4761.219243,2773.533796,7873.760349,0.366400,-202.233444,11.503313,40.455971,18.475946,...,12.299253,3.442242,14.397862,2.564635,8.012366,-0.348381,6.556405,0.761705,1.236786,AtlanticSpottedDolphin
1,0.556443,0.034761,4708.987505,2958.766276,8038.454217,0.332751,-223.538940,17.380117,39.020512,21.792746,...,13.133816,2.862710,9.231984,1.513073,6.790045,0.612614,2.225558,2.136981,0.256718,AtlanticSpottedDolphin
2,0.528642,0.019785,5044.704559,3117.567829,8513.897236,0.353713,-259.795319,17.206076,41.884102,16.008436,...,8.326727,5.092938,9.672286,0.974509,8.395206,-3.988970,4.246128,-3.778867,4.461563,AtlanticSpottedDolphin
3,0.456285,0.024098,4600.430300,2970.604537,7999.978861,0.320586,-273.871765,26.248457,41.347931,16.910347,...,13.801317,0.085264,10.640569,2.333693,6.013561,3.245038,1.631215,1.111101,0.356827,AtlanticSpottedDolphin
4,0.496730,0.021204,5286.915839,2987.326793,8453.739790,0.442827,-246.035934,8.909916,39.739197,14.844665,...,12.734055,-0.698437,9.860725,0.086621,4.310746,-0.915217,2.149506,-2.085563,0.539603,AtlanticSpottedDolphin


In [26]:
# Feature extraction
class_list = df.iloc[:,-1]
print(class_list)
encoder = LabelEncoder()
y = encoder.fit_transform(class_list)
print("y: ", y)
input_parameters = df.iloc[:, 0:26]
scaler = StandardScaler()
X = scaler.fit_transform(np.array(input_parameters))
print("X:", X)

0        AtlanticSpottedDolphin
1        AtlanticSpottedDolphin
2        AtlanticSpottedDolphin
3        AtlanticSpottedDolphin
4        AtlanticSpottedDolphin
                  ...          
13077                SpermWhale
13078                SpermWhale
13079                SpermWhale
13080                SpermWhale
13081                SpermWhale
Name: label, Length: 13082, dtype: object
y:  [ 0  0  0 ... 44 44 44]
X: [[ 0.04904231  0.65321755  0.8995295  ...  0.71445222  0.50887267
  -0.41604051]
 [ 0.16167761  0.01840228  0.86789605 ... -0.09450453  0.78695808
  -0.59984215]
 [-0.03241632 -0.37603894  1.07121853 ...  0.28291678 -0.4092459
   0.18873359]
 ...
 [ 0.55866138 -0.57543748  0.71099637 ... -1.30726783 -0.46502046
  -0.86291154]
 [-1.29921903  0.10527015  0.88070438 ...  1.29560364  0.69039726
   0.06387908]
 [ 0.48617552  0.01764527  0.43533272 ...  0.19980501 -0.52474238
   0.12223975]]


In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
#normalizer = Normalization(axis=-1)
#normalizer.adapt(X_train)

In [29]:
#Setup model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation = 'relu', input_shape = (X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(45, activation = 'softmax'),
])

print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                │ (None, 512)            │        13,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 45)             │         2,925 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,229 (739.18 KB)

 Trainable params: 189,229 (739.18 KB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

def trainModel(model,epochs, optimizer):
    batch_size = 128
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epochs, batch_size = batch_size, callbacks=[tensorboard_callback])

In [31]:
#%tensorboard --logdir logs
model_history = trainModel(model = model, epochs = 75, optimizer = 'adam')

Epoch 1/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.3112 - loss: 2.7657 - val_accuracy: 0.6439 - val_loss: 1.2388
Epoch 2/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6374 - loss: 1.3065 - val_accuracy: 0.7509 - val_loss: 0.8863
Epoch 3/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7263 - loss: 0.9831 - val_accuracy: 0.8013 - val_loss: 0.7097
Epoch 4/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7714 - loss: 0.8057 - val_accuracy: 0.8334 - val_loss: 0.5734
Epoch 5/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8055 - loss: 0.6929 - val_accuracy: 0.8491 - val_loss: 0.5099
Epoch 6/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8242 - loss: 0.5996 - val_accuracy: 0.8590 - val_loss: 0.4719
Epoch 7/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8444 - loss: 0.5476 - val_accuracy: 0.8835 - val_loss: 0.4011
Epoch 8/75
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8453 - loss: 0.5141 - val_accuracy: 0.8888 - v

In [51]:
#os.makedirs(f'/kaggle/working/model_versions/acoustic_classification4', exist_ok=True)
model.save(f'/kaggle/working/model_versions/acoustic_classification6.keras')
joblib.dump(scaler, "/kaggle/working/model_versions/scaler") 

['/kaggle/working/model_versions/scaler']

In [47]:
from IPython.display import FileLink
#!cd /kaggle/working/model_versions/
!ls
!zip -r file.zip model_versions/acoustic_classification5
FileLink('file.zip')

data.csv  file.zip  logs  model_versions


/kaggle/working/file.zip